In [5]:
import pandas as pd

df = pd.read_csv("cleaned_data.csv")

In [6]:
df

,Unnamed: 0.1,grant_key,Unnamed: 0,id,agency,awardeeName,awardeeCity,awardeeStateCode,piFirstName,piLastName,...,date,startDate,expDate,fundsObligatedAmt,abstractText,orgCandidate,_org,org_combo_flag,org_count_flag,is_multi_org_flag
0,0,id:1002878,1396,1002878,NSF,University of Connecticut Health Center,FARMINGTON,CT,Raquell,Holmes,...,2010-09-08,2010-09-01,2012-08-31,98058,Improvisational Theater for Computing Scientis...,Beaumont Health,Beaumont,Beaumont & Corewell & Spectrum,3,True
1,1,id:1004847,907,1004847,NSF,University of North Texas Health Science Cente...,FORT WORTH,TX,Rita,Patterson,...,2010-04-12,2010-04-15,2011-03-31,20000,1004847 Patterson The Summer Bioengineerin...,Spectrum Health,Spectrum,Beaumont & Corewell & Spectrum,3,True
2,2,id:1006950,403,1006950,NSF,University of New Mexico Health Sciences Center,ALBUQUERQUE,NM,Stephanie,Ruby,...,2010-01-04,2010-02-01,2011-01-31,30000,Intellectual merit. Spliceosomal RNAs are thou...,Corewell Health,Corewell,Beaumont & Corewell & Spectrum,3,True
3,3,id:1007261,375,1007261,NSF,Health Research Incorporated/New York State De...,MENANDS,NY,Liaquat,Husain,...,2010-09-16,2010-10-01,2015-03-31,273917,Particles are important in the global climate ...,Corewell Health,Corewell,Beaumont & Corewell & Spectrum,3,True
4,4,id:1009244,1386,1009244,NSF,Florida Department of Health Division of Infor...,TALLAHASSEE,FL,Kendra,Goff,...,2010-09-29,2010-10-01,2015-09-30,143032,"Project Abstract Around the world, harmful al...",Beaumont Health,Beaumont,Beaumont & Corewell & Spectrum,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,508,id:964517,412,964517,NSF,Louisiana State University Health Sciences Center,NEW ORLEANS,LA,Siqiong,Liu,...,2010-05-04,2009-09-01,2011-08-31,55168,The long-term goal of this project is to revea...,Corewell Health,Corewell,Beaumont & Corewell & Spectrum,3,True
509,509,id:964613,1392,964613,NSF,The University of Texas Health Science Center ...,HOUSTON,TX,Elmer,Bernstam,...,2010-09-07,2010-09-01,2015-08-31,599607,A clinical data warehouse (CDW) is a repositor...,Beaumont Health,Beaumont,Beaumont & Corewell & Spectrum,3,True
510,510,id:964728,912,964728,NSF,University of Toledo Health Science Campus,TOLEDO,OH,Robert,Blumenthal,...,2010-03-03,2010-03-01,2013-08-31,569999,Intellectual Merit: The biosphere is dominate...,Spectrum Health,Spectrum,Beaumont & Corewell & Spectrum,3,True
511,511,id:966482,939,966482,NSF,"Loyola University of Chicago, Health Sciences ...",MAYWOOD,IL,Charles,"Webber, Jr",...,2009-12-04,2009-01-11,2012-12-31,38132,How well can you walk and talk at the same tim...,Spectrum Health,Spectrum,Beaumont & Corewell & Spectrum,3,True


In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# =========================
# Helpers: build flags if missing, coerce types, dedupe
# =========================
def normalize_org(text: str) -> str | None:
    t = str(text).upper()
    if "SPECTRUM" in t: return "Spectrum"
    if "BEAUMONT" in t or "BEAUMON" in t: return "Beaumont"
    if "COREWELL" in t: return "Corewell"
    return None

def make_grant_key(r):
    v = r.get("id")
    if pd.notnull(v):
        try: return f"id:{int(v)}"
        except Exception: pass
    return f"title:{r.get('title','')[:150]}|start:{r.get('startDate')}|exp:{r.get('expDate')}|funds:{r.get('fundsObligatedAmt')}"

def ensure_flags(df: pd.DataFrame) -> pd.DataFrame:
    # Parse dates
    for c in ["date","startDate","expDate"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")

    # grant_key
    if "grant_key" not in df.columns:
        df["grant_key"] = df.apply(make_grant_key, axis=1)

    # If flags exist, coerce types; else compute
    have_combo = "org_combo_flag" in df.columns
    have_count = "org_count_flag" in df.columns
    have_multi = "is_multi_org_flag" in df.columns

    if have_combo and have_count and have_multi:
        # Coerce types from strings like '3' and 'TRUE'
        df["org_count_flag"] = pd.to_numeric(df["org_count_flag"], errors="coerce").fillna(0).astype(int)
        df["is_multi_org_flag"] = df["is_multi_org_flag"].astype(str).str.upper().map({"TRUE": True, "FALSE": False}).fillna(df["org_count_flag"] > 1)
    else:
        # Compute from scratch using the three orgs only
        df["_org"] = df.apply(
            lambda r: normalize_org(f"{r.get('orgCandidate','')} {r.get('awardeeName','')} {r.get('title','')}"),
            axis=1
        )
        combo = (
            df.groupby("grant_key")["_org"]
              .apply(lambda s: sorted([o for o in set(s) if o]))  # keep only Corewell/Spectrum/Beaumont
              .reset_index()
        )
        def list_to_flag(orgs): return " & ".join(orgs) if orgs else "Other"
        combo["org_combo_flag"] = combo["_org"].apply(list_to_flag)
        combo["org_count_flag"] = combo["_org"].apply(len).astype(int)
        combo["is_multi_org_flag"] = combo["org_count_flag"] > 1
        combo = combo.drop(columns=["_org"])
        df = df.merge(combo, on="grant_key", how="left")

    # Deduplicate to one row per grant (first wins to avoid fund double-count)
    df_simple = (
        df.drop(columns=[c for c in ["_org","Unnamed: 0"] if c in df.columns], errors="ignore")
          .sort_values("grant_key")
          .groupby("grant_key", as_index=False)
          .first()
    )

    # Numeric funds
    if "fundsObligatedAmt" in df_simple.columns:
        df_simple["fundsObligatedAmt"] = pd.to_numeric(df_simple["fundsObligatedAmt"], errors="coerce")

    # Fill missing flags
    df_simple["org_combo_flag"] = df_simple["org_combo_flag"].fillna("Other")
    df_simple["org_count_flag"] = df_simple["org_count_flag"].fillna(0).astype(int)
    df_simple["is_multi_org_flag"] = df_simple["is_multi_org_flag"].fillna(df_simple["org_count_flag"] > 1)

    return df_simple

# >>> Run after you've read your df
df_simple = ensure_flags(df)

# =========================
# Visualizations
# =========================
figs = []

# A) By org_combo_flag
combo_agg = (
    df_simple.groupby("org_combo_flag", as_index=False)
             .agg(total_funds=("fundsObligatedAmt","sum"),
                  n_grants=("grant_key","count"))
             .sort_values("total_funds", ascending=False)
)
fig_combo_funds = px.bar(
    combo_agg, x="org_combo_flag", y="total_funds", text="total_funds",
    title="Total Funds by org_combo_flag",
    labels={"org_combo_flag":"","total_funds":"Total Funds ($)"}
)
fig_combo_funds.update_traces(texttemplate="%{text:,}", textposition="outside")
figs.append(("Total Funds by org_combo_flag", fig_combo_funds))

fig_combo_counts = px.bar(
    combo_agg, x="org_combo_flag", y="n_grants", text="n_grants",
    title="Grant Count by org_combo_flag",
    labels={"org_combo_flag":"","n_grants":"# Grants"}
)
fig_combo_counts.update_traces(textposition="outside")
figs.append(("Grant Count by org_combo_flag", fig_combo_counts))

fig_combo_box = px.box(
    df_simple, x="org_combo_flag", y="fundsObligatedAmt",
    title="Funds Distribution by org_combo_flag",
    labels={"org_combo_flag":"","fundsObligatedAmt":"Funds ($)"}
)
figs.append(("Funds Distribution by org_combo_flag", fig_combo_box))

# B) By org_count_flag (1 / 2 / 3)
count_agg = (
    df_simple.groupby("org_count_flag", as_index=False)
             .agg(total_funds=("fundsObligatedAmt","sum"),
                  n_grants=("grant_key","count"))
             .sort_values("org_count_flag")
)
fig_count_funds = px.bar(
    count_agg, x="org_count_flag", y="total_funds", text="total_funds",
    title="Total Funds by org_count_flag (1 / 2 / 3)",
    labels={"org_count_flag":"org_count_flag","total_funds":"Total Funds ($)"}
)
fig_count_funds.update_traces(texttemplate="%{text:,}", textposition="outside")
figs.append(("Total Funds by org_count_flag", fig_count_funds))

fig_count_counts = px.bar(
    count_agg, x="org_count_flag", y="n_grants", text="n_grants",
    title="Grant Count by org_count_flag (1 / 2 / 3)",
    labels={"org_count_flag":"org_count_flag","n_grants":"# Grants"}
)
fig_count_counts.update_traces(textposition="outside")
figs.append(("Grant Count by org_count_flag", fig_count_counts))

fig_count_pie = px.pie(
    df_simple, names="org_count_flag", values="fundsObligatedAmt",
    title="Share of Funds by org_count_flag"
)
figs.append(("Share of Funds by org_count_flag", fig_count_pie))

# C) Individual grants only (is_multi_org_flag == False)
indiv = df_simple[df_simple["is_multi_org_flag"] == False].copy()

if not indiv.empty:
    if {"startDate","expDate","awardeeName"}.issubset(indiv.columns):
        fig_indiv_timeline = px.timeline(
            indiv, x_start="startDate", x_end="expDate", y="awardeeName",
            color="org_combo_flag",
            hover_data=["title","fundsObligatedAmt","org_combo_flag"],
            title="Individual Grants — Timelines"
        )
        fig_indiv_timeline.update_yaxes(autorange="reversed")
        figs.append(("Individual Grants — Timelines", fig_indiv_timeline))

    if {"startDate","fundsObligatedAmt"}.issubset(indiv.columns):
        fig_indiv_scatter = px.scatter(
            indiv, x="startDate", y="fundsObligatedAmt",
            color="org_combo_flag", hover_name="title",
            title="Individual Grants — Start Date vs Funds",
            labels={"fundsObligatedAmt":"Funds ($)","startDate":"Start Date"}
        )
        figs.append(("Individual Grants — Start vs Funds", fig_indiv_scatter))

    if {"awardeeName","fundsObligatedAmt"}.issubset(indiv.columns):
        fig_indiv_tree = px.treemap(
            indiv, path=["org_combo_flag","awardeeName"],
            values="fundsObligatedAmt",
            title="Individual Grants — Treemap (org_combo_flag → awardee)"
        )
        figs.append(("Individual Grants — Treemap", fig_indiv_tree))

    def _fmt_money(s):
        return s.fillna(0).astype(float).map(lambda v: f"${v:,.0f}")
    fig_indiv_table = go.Figure(data=[go.Table(
        header=dict(values=["Grant Key","Title","Start","End","Funds","Combo"],
                    fill_color="#e5ecf6", align="left"),
        cells=dict(values=[
            indiv.get("grant_key"),
            indiv.get("title"),
            indiv.get("startDate").dt.date.astype(str) if "startDate" in indiv else pd.Series([""]*len(indiv)),
            indiv.get("expDate").dt.date.astype(str) if "expDate" in indiv else pd.Series([""]*len(indiv)),
            _fmt_money(indiv.get("fundsObligatedAmt")),
            indiv.get("org_combo_flag"),
        ], align="left")
    )])
    fig_indiv_table.update_layout(title="Individual Grants — Table")
    figs.append(("Individual Grants — Table", fig_indiv_table))
else:
    note = go.Figure()
    note.add_annotation(text="No individual grants found (is_multi_org_flag == False).",
                        xref="paper", yref="paper", x=0.5, y=0.5, showarrow=False)
    note.update_layout(title="Individual Grants — None")
    figs.append(("Individual Grants — None", note))

# =========================
# Save one HTML
# =========================
out_path = "org_combo_and_individual_grants.html"
with open(out_path, "w", encoding="utf-8") as f:
    f.write("""<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8">
<title>Org Combo & Individual Grants — Plotly Report</title>
<style>
body{font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu;max-width:1100px;margin:2rem auto;padding:0 1rem}
h1{margin-top:0} h2{margin:2rem 0 .75rem}
</style></head><body>
<h1>Org Combo & Individual Grants — Plotly Report</h1>
""")
    for i,(title,fig) in enumerate(figs):
        f.write(f"<h2>{i+1}. {title}</h2>")
        f.write(fig.to_html(full_html=False, include_plotlyjs=("cdn" if i==0 else False)))
        f.write("<hr/>")
    f.write("</body></html>")

print(f"Saved {out_path}")

Saved org_combo_and_individual_grants.html


In [12]:
import pandas as pd
import plotly.express as px

# Assumes df_simple has 'startDate' (or 'date') and 'org_combo_flag'
df_plot = df_simple.copy()

# --- pick a date column ---
for c in ["startDate", "date"]:
    if c in df_plot.columns:
        df_plot[c] = pd.to_datetime(df_plot[c], errors="coerce")
date_col = "startDate" if "startDate" in df_plot.columns else "date"

# --- extract year ---
df_plot = df_plot.dropna(subset=[date_col])
df_plot["year"] = df_plot[date_col].dt.year
df_plot["org_combo_flag"] = df_plot["org_combo_flag"].fillna("Other")

# --- aggregate: count grants per year & combo ---
counts = (
    df_plot.groupby(["year","org_combo_flag"], as_index=False)
           .agg(n_grants=("grant_key","count"))
)

# --- line plot ---
fig = px.line(
    counts,
    x="year",
    y="n_grants",
    color="org_combo_flag",
    markers=True,
    title="Number of Grants per Year by Org Combo",
    labels={"year":"Year","n_grants":"# Grants","org_combo_flag":"Org Combo"}
)

fig.update_layout(
    xaxis=dict(dtick=1),  # show every year
    xaxis_title="Year",
    yaxis_title="# Grants",
    legend_title="Org Combo"
)

fig.show()

# optional: save HTML
with open("grants_per_year_by_org_combo.html","w",encoding="utf-8") as f:
    f.write(fig.to_html(full_html=True, include_plotlyjs="cdn"))

print("Saved grants_per_year_by_org_combo.html")

Saved grants_per_year_by_org_combo.html
